In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from torch.utils.data import Dataset
import os
import copy
from tensorboardX import SummaryWriter
from PIL import Image
import torchvision.transforms as transforms

plt.ion()   # interactive mode
%matplotlib inline

In [9]:
class CustomDataset(Dataset):
    def __init__(self, root='D:\\Research\\data\\UCMerced_LandUse\\Images', transforms_=None, idx=[]):
        self.transform = transforms_
        self.files = []

        for path, subdirs, files in os.walk(root):
            for name in files:
                self.files.append(os.path.join(path, name))
        
        self.files =  [self.files[i] for i in idx]
        self.n = len(self.files)
        
        classes = os.listdir(root)
        self.classes = {}
        
        for i, cls in enumerate(classes):
            self.classes[cls] = i
        print(self.classes)


    def __getitem__(self, index):
        filepath = self.files[index % len(self.files)]

        cls_name = filepath.split('\\')[-2]
        img = self.transform(Image.open(filepath))
        label = self.classes[cls_name]

        return img, label

    def __len__(self):
        return len(self.files)

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

class_names = 21
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_sizes = {'val': 210, 'train': 2100-210}


In [4]:
from tqdm import tqdm

def train_model(model, criterion, optimizer, scheduler, dataloaders, writer=None, i=0,  num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

            writer.add_scalar(phase+"/loss", epoch_loss, epoch)
            writer.add_scalar(phase+"/acc", epoch_acc, epoch)


            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                # torch.save(model.state_dict(), 'saved/resnet_'+str(i)+'.pt')

        print()
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

# Resnet 152

In [5]:
accuracies = []
np.random.seed(0)
index = list(np.random.permutation(2100))


for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=4,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet152(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")

Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [02:51<00:00,  1.44s/it]


train Loss: 2.0814 Acc: 0.5365


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:10<00:00,  1.31it/s]


val Loss: 0.9571 Acc: 0.8619

Epoch 1/19
----------


...

train Loss: 0.0752 Acc: 0.9905


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  4.84it/s]


val Loss: 0.0582 Acc: 0.9905

Epoch 19/19
----------


  3%|██▊                                                                               | 4/119 [00:02<01:19,  1.45it/s]


KeyboardInterrupt: 

In [7]:
torch.save(model_ft.state_dict(), 'saved/resnet_152.pt')

In [ ]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

In [ ]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)

# Resnet 101


In [6]:
np.random.seed(0)
index = list(np.random.permutation(2100))

for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet101(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")
    
torch.save(model_ft.state_dict(), 'saved/resnet_101.pt')

Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [01:06<00:00,  3.59it/s]


train Loss: 1.7444 Acc: 0.5519


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00, 10.85it/s]


val Loss: 0.6774 Acc: 0.8952

Epoch 1/19
----------


  1%|▋                                                                                 | 2/237 [00:00<01:05,  3.60it/s]

KeyboardInterrupt: 

In [10]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}



100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:08<00:00, 23.95it/s]


[[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 11, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0, 0],
 [0

In [32]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)


Row Sum: 
 [10  9 14 10 13 11  3 13 10  4 10  5  7 12 14 17  6  8  9 13 12]

Col Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 15  3 11  5 17  7  8  9 12 12]

Precision
 [1.         1.         1.         0.9        1.         1.
 1.         0.84615385 0.9        1.         1.         1.
 0.42857143 0.91666667 0.35714286 1.         0.83333333 1.
 1.         0.92307692 1.        ]

Recall
 [1.         1.         1.         0.69230769 1.         1.
 0.6        1.         1.         0.66666667 1.         0.33333333
 1.         1.         1.         1.         0.71428571 1.
 1.         1.         1.        ]

Model average prcision:  0.909759288330717
Model average recall:  0.9050758765044478
Model accuracy:  0.9047619047619048


# Resnet 50

In [ ]:
np.random.seed(0)
index = list(np.random.permutation(2100))

for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")
    
torch.save(model_ft.state_dict(), 'saved/resnet_50.pt')

In [33]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}



100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:08<00:00, 23.61it/s]



Row Sum: 
 [10  9 14 10 13 11  3 13 10  4 10  5  7 12 14 17  6  8  9 13 12]

Col Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 15  3 11  5 17  7  8  9 12 12]

Precision
 [1.         1.         1.         0.9        1.         1.
 1.         0.84615385 0.9        1.         1.         1.
 0.42857143 0.91666667 0.35714286 1.         0.83333333 1.
 1.         0.92307692 1.        ]

Recall
 [1.         1.         1.         0.69230769 1.         1.
 0.6        1.         1.         0.66666667 1.         0.33333333
 1.         1.         1.         1.         0.71428571 1.
 1.         1.         1.        ]

Model average prcision:  0.909759288330717
Model average recall:  0.9050758765044478
Model accuracy:  0.9047619047619048


In [ ]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)

# Resnet 34

In [ ]:
np.random.seed(0)
index = list(np.random.permutation(2100))

for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet34(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")
    
torch.save(model_ft.state_dict(), 'saved/resnet_34.pt')

In [ ]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

In [ ]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)

# Resnet 18

In [ ]:
np.random.seed(0)
index = list(np.random.permutation(2100))

for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")
    
torch.save(model_ft.state_dict(), 'saved/resnet_18.pt')

In [ ]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

In [ ]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)